In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Задача №1

Дана матрица объект-признак
```
X = [[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]
```
и значения целевой переменной
```
y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
      7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]
```

Подберите два признака (из четырёх) так, чтобы уровень линейной зависимости целевой переменной от значений этих признаков был максимальным. Другими словами, модель линейной регрессии на этих признаках должна давать наилучший результат. В качестве ответа запишите значение коэффициента детерминации для модели на этих признаках.

In [18]:
import numpy as np
import math
import scipy.stats as st
from itertools import combinations

In [20]:
X = [[ 1.22401313,  2.30868478,  3.03636353,  2.69287214],
     [-0.18757272,  1.30337355,  5.12093014,  3.46363202],
     [-0.81094525,  1.82463398,  5.79686488,  1.86159445],
     [ 0.75129018,  2.67392052,  3.65529809,  1.66746094],
     [ 0.00972362,  1.97367255,  2.50594319,  1.69755173],
     [-0.62972637,  0.77750764,  2.84124027,  4.54410559],
     [ 2.29536229,  1.81206697,  1.95026215,  1.51874636],
     [ 0.0920418 ,  2.26971361,  7.47708735,  2.61081203],
     [ 2.39252799,  3.17563985,  3.61420599,  5.10773362],
     [ 0.54983815,  2.87988651,  1.65752765,  1.59635987]]

y = [ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
      7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218]
X = np.array(X)
y = np.array(y)

def R2(x:np.ndarray, y:np.ndarray):
    
    ones = np.ones((x.shape[0],1))
    x = np.hstack([ones, x])
    
    XTX = x.T.dot(x)
    XTX_inv = np.linalg.inv(XTX)
    
    b = XTX_inv.dot(x.T).dot(y)
    
    z = x.dot(b)
    e = y - z
    
    return 1 - e.var() / y.var()

kd=[]
for i, j in combinations(range(X.shape[1]), 2):
    r = R2(X[:, [i,j]], y)
    print(i, j, r)
    kd.append(r)

print("Ответ: ",round(max(kd),8))

### Задача №2 

Является ли значимым уравнение регрессии, полученное в предыдущей задаче? В качестве ответа запишите 1, если является, и 0 иначе.

In [21]:
k1 = 2
k2 = X.shape[0]- 2 - 1
RR2 = R2(X[:, [0,2]], y)
F = (RR2 / k1) / ((1 - RR2) / k2)
alpha = 0.05
t = st.f.ppf(1 - alpha, k1, k2)
print("Ответ: ",int(F>=t))

### Задача №3

Для проведения A/B-тестирования сайта интернет-магазина были получены следующие данные: страница `A` была посещена 2509 раз, из них 77 закончились совершением покупки, страница `B` была посещена 1465 раз, 60 из них закончились совершением покупки. Является ли значимым отличие конверсии на страницах `A` и `B`? В качестве ответа запишите 1, если является, и 0 иначе.

_Подсказка_. Реализуйте двухвыборочный t-тест. В качестве выборок здесь можно взять наборы меток совершения покупки (0 или 1) каждым посетителем.

In [22]:
y1 = np.zeros(2509)
y1[np.arange(77)] = 1
y2 = np.zeros(1465)
y2[np.arange(60)] = 1
y = np.concatenate([y1, y2])
n1 = y1.shape[0]
n2 = y2.shape[0]
n = n1+n2
y1_mean = y1.mean()
y2_mean = y2.mean()
y_mean = y.mean()
SSb = n1*(y1_mean - y_mean)**2 + n2*(y2_mean - y_mean)**2
SSw = ((y1 - y1_mean)**2).sum() + ((y2 - y2_mean)**2).sum()
k = 2
k1 = k-1
k2 = n-k
sigma2b = SSb/k1
sigma2w = SSw/k2
F = sigma2b/sigma2w
alpha = 0.05
t = st.f.ppf(1 - alpha, k1, k2)
print("Ответ: ",int(F>=t))